In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

In [2]:
# Diretórios
folder = '../output/'
output_dir = 'figures_metrics'
os.makedirs(output_dir, exist_ok=True)

In [3]:
# Mapeamento de rótulos para nomes amigáveis
modelo_nomes = {
    'BLR': 'Bayesian Linear Regressor',
    'EWA': 'EWARegressor',
    'KNN': 'KNNRegressor',
    'SKL': 'Scikit-Learn Regressor',
    'SRP': 'SRPRegressor'
    # adicione mais se quiser
}

In [4]:
# Detectar modelos presentes nos arquivos
arquivos_modelo = [f for f in os.listdir(folder) if not f.startswith('Ideal_') and f.endswith('.csv')]
modelos_detectados = sorted(set(re.match(r'^([A-Za-z0-9_]+)_\d+_', f).group(1) for f in arquivos_modelo if re.match(r'^([A-Za-z0-9_]+)_\d+_', f)))
print("Modelos detectados:", modelos_detectados)

# Detectar intervalos de cluster
ideal_files = [f for f in os.listdir(folder) if f.startswith('Ideal_') and f.endswith('.csv')]
i_range = sorted({int(re.search(r'_(\d+)', f).group(1)) for f in ideal_files})

Modelos detectados: ['BLR', 'EWA', 'KNN', 'SKL', 'SRP']


In [5]:
file_config = {
    "unallocated_requests": {
        "column_y": "Unallocated Requests"
    },
    "active_amfs": {
        "column_y": "Active AMFs"
    },
    "amf_utilization": {
        "column_y": "AMF Utilization (%)"
    }
}

In [6]:
def calculate_metrics_with_ideal(file_config, i_range):
    summary = []

    for key, config in file_config.items():
        for i in i_range:
            csv_files = {
                "ideal": os.path.join(folder, f"Ideal_{i}_{key}.csv".replace("active_amfs", "active_amfs_log"))
            }

            for model in modelos_detectados:
                model_path = os.path.join(folder, f"{model}_{i}_{key}.csv".replace("active_amfs", "active_amfs_log"))
                csv_files[model] = model_path

            data_frames = {}
            for label, path in csv_files.items():
                if not os.path.exists(path):
                    print(f"[Aviso] Arquivo não encontrado: {path}")
                    continue

                df = pd.read_csv(path)
                if config["column_y"] not in df.columns:
                    print(f"[Erro] Coluna '{config['column_y']}' ausente em {path}")
                    continue

                data_frames[label] = df

            if len(data_frames) < len(modelos_detectados) + 1:
                print(f"[Pular] Cluster {i}: dados incompletos para '{key}'")
                continue

            ideal_sum = data_frames["ideal"][config["column_y"]].sum()

            for label, df in data_frames.items():
                if label == "ideal":
                    continue

                total = df[config["column_y"]].sum()
                average = df[config["column_y"]].mean()
                max_val = df[config["column_y"]].max()
                deviation = total - ideal_sum
                deviation_pct = (deviation / ideal_sum) * 100 if ideal_sum != 0 else None

                summary.append({
                    "Cluster": i,
                    "Metric": key,
                    "Model": modelo_nomes.get(label, label),
                    "Total": total,
                    "Average": average,
                    "Max": max_val,
                    "Ideal Total": ideal_sum,
                    "Deviation": deviation,
                    "Deviation (%)": deviation_pct
                })

    return pd.DataFrame(summary)

# Executar e salvar
metrics_summary = calculate_metrics_with_ideal(file_config, i_range)
summary_path = os.path.join(output_dir, "metrics_summary_with_ideal.csv")
metrics_summary.to_csv(summary_path, index=False)
print(f"Métricas salvas em: {summary_path}")

Métricas salvas em: figures_metrics/metrics_summary_with_ideal.csv


Atendidas

In [10]:
# Diretórios
folder = '../output/'
output_folder = 'figures_attended'
os.makedirs(output_folder, exist_ok=True)

# Detecta automaticamente os modelos disponíveis
arquivos = os.listdir(folder)
modelo_pattern = re.compile(r"^([A-Za-z0-9_]+)_\d+_unallocated_requests\.csv$")
modelos = sorted({match.group(1) for f in arquivos if (match := modelo_pattern.match(f))})
print("Modelos detectados:", modelos)

# Rótulos opcionais para exibição
modelo_nomes = {
    'BLR': 'Bayesian Linear Regressor',
    'EWA': 'EWARegressor',
    # outros modelos aqui, se quiser
}
rotulos = [modelo_nomes.get(m, m) for m in modelos]

# Intervalo de experimentos baseado nos arquivos
exp_nums = sorted({int(re.search(r'_(\d+)_', f).group(1)) for f in arquivos if 'unallocated_requests.csv' in f})
print("Clusters detectados:", exp_nums)

# Tempo
start_time = pd.to_datetime("2013-12-01 00:00")
freq = "10min"

# Total de perdas para tabela final
tabela_perdas = []

def extract_total_requests(log_path):
    totals = []
    with open(log_path, 'r') as f:
        for line in f:
            if line.startswith("Time"):
                match = re.search(r'Total Requests: (\d+)', line)
                if match:
                    totals.append(int(match.group(1)))
    return pd.Series(totals)

def plot_attended_comparison(exp_num):
    dfs_unallocated = {}
    dfs_total = {}
    time_index = None

    for modelo in modelos:
        unallocated_file = f'{folder}{modelo}_{exp_num}_unallocated_requests.csv'
        log_file = f'{folder}{modelo}_{exp_num}_states_log.txt'

        if not os.path.exists(unallocated_file) or not os.path.exists(log_file):
            print(f"[Aviso] Arquivos ausentes para modelo {modelo}, experimento {exp_num}.")
            continue

        df_unallocated = pd.read_csv(unallocated_file)
        df_total = extract_total_requests(log_file)

        if time_index is None:
            time_index = pd.date_range(start=start_time, periods=len(df_unallocated), freq=freq)

        df_unallocated.index = time_index
        df_total.index = time_index

        dfs_unallocated[modelo] = df_unallocated
        dfs_total[modelo] = df_total

    if not dfs_unallocated:
        print(f"[Pular] Nenhum dado disponível para cluster {exp_num}")
        return

    plt.figure(figsize=(14, 8))

    for modelo in dfs_unallocated:
        pct_attended = (dfs_total[modelo] - dfs_unallocated[modelo]['Unallocated Requests']) / dfs_total[modelo] * 100
        plt.plot(time_index, pct_attended, label=modelo_nomes.get(modelo, modelo), marker='o', alpha=0.6)

        # Adiciona ao resumo total de perdas
        total_loss = dfs_unallocated[modelo]['Unallocated Requests'].sum()
        eventos_perda = (dfs_unallocated[modelo]['Unallocated Requests'] > 0).sum()
        tabela_perdas.append({
            "Cluster": exp_num,
            "Modelo": modelo_nomes.get(modelo, modelo),
            "Total de Requisições Perdidas": total_loss,
            "Eventos de Perda": eventos_perda
        })

    plt.xlabel("Tempo", fontsize=14)
    plt.ylabel("Requisições Atendidas (%)", fontsize=14)
    plt.title(f'Cluster {exp_num} - Porcentagem de Requisições Atendidas', fontsize=16)
    plt.ylim(0, 105)
    plt.grid(True)
    plt.legend(fontsize=12)

    tick_spacing = len(time_index) // 10
    plt.xticks(ticks=time_index[::tick_spacing],
               labels=time_index[::tick_spacing].strftime('%Y-%m-%d %H:%M'),
               rotation=45, ha='right')

    plt.tight_layout()
    plt.savefig(f'{output_folder}/Cluster_{exp_num}_attended_comparison.png')
    plt.close()

# Executa para todos os clusters encontrados
for i in exp_nums:
    plot_attended_comparison(i)

# Exporta a tabela final com perdas
df_perdas = pd.DataFrame(tabela_perdas)
perdas_csv = os.path.join(output_folder, "tabela_requisicoes_perdidas_por_cluster.csv")
df_perdas.to_csv(perdas_csv, index=False)

print(f"\n✅ Gráficos salvos em '{output_folder}' e tabela de perdas salva em:\n→ {perdas_csv}")

Modelos detectados: ['BLR', 'EWA', 'Ideal', 'KNN', 'SKL', 'SRP']
Clusters detectados: [0, 1, 2, 3, 4]

✅ Gráficos salvos em 'figures_attended' e tabela de perdas salva em:
→ figures_attended/tabela_requisicoes_perdidas_por_cluster.csv


In [3]:
import pandas as pd
import os
import re

folder = '../output/'
file_config = {
    "ON_amfs": {
        "column_y": "ON AMFs"
    }
}

# Detectar modelos automaticamente a partir dos arquivos (excluindo Ideal)
arquivos = os.listdir(folder)
modelos_detectados = sorted({f.split('_')[0] for f in arquivos if f.endswith('ON_amfs_log.csv') and not f.startswith('Ideal_')})
rotulos = modelos_detectados  # você pode mapear nomes se quiser
i_range = sorted({int(f.split('_')[1]) for f in arquivos if f.startswith('Ideal_') and 'ON_amfs' in f})

def calcular_erros_apenas(file_config, i_range, modelos):
    erros = []

    for key, config in file_config.items():
        for i in i_range:
            arquivos_csv = {"ideal": os.path.join(folder, f"Ideal_{i}_{key}.csv".replace("ON_amfs", "ON_amfs_log"))}
            for modelo in modelos:
                arquivos_csv[modelo] = os.path.join(folder, f"{modelo}_{i}_{key}.csv".replace("ON_amfs", "ON_amfs_log"))

            dfs = {}
            for nome, caminho in arquivos_csv.items():
                if os.path.exists(caminho):
                    df = pd.read_csv(caminho)
                    if config["column_y"] in df.columns:
                        dfs[nome] = df
                    else:
                        print(f"[Erro] Coluna ausente em {caminho}")
                else:
                    print(f"[Aviso] Arquivo não encontrado: {caminho}")

            if len(dfs) < len(modelos) + 1:
                print(f"[Pular] Cluster {i}: dados incompletos")
                continue

            ideal_vals = dfs["ideal"][config["column_y"]].tolist()
            min_len = min(len(ideal_vals), *(len(dfs[m][config["column_y"]]) for m in modelos))

            for t in range(min_len):
                linha_erro = {
                    "Cluster": i,
                    "Tempo": t,
                    "Ideal": ideal_vals[t]
                }
                for modelo in modelos:
                    val = dfs[modelo][config["column_y"]][t]
                    erro = val - ideal_vals[t]
                    linha_erro[f"{modelo}"] = val
                    linha_erro[f"Erro ({modelo})"] = erro
                erros.append(linha_erro)
    return pd.DataFrame(erros)

# Executar e salvar resultado
df_erros = calcular_erros_apenas(file_config, i_range, modelos_detectados)
df_erros.to_csv("relacao_erros_ON_amfs.csv", index=False)
print("Relação de erros salva em: relacao_erros_ON_amfs.csv")


Relação de erros salva em: relacao_erros_ON_amfs.csv


In [4]:
import pandas as pd
import os
import re

folder = '../output/'
file_config = {
    "ON_amfs": {
        "column_y": "ON AMFs"
    }
}

# Detectar modelos automaticamente
arquivos = os.listdir(folder)
modelos_detectados = sorted({f.split('_')[0] for f in arquivos if f.endswith('ON_amfs_log.csv') and not f.startswith('Ideal_')})
i_range = sorted({int(f.split('_')[1]) for f in arquivos if f.startswith('Ideal_') and 'ON_amfs' in f})

def calcular_erros(file_config, i_range, modelos):
    erros = []

    for key, config in file_config.items():
        for i in i_range:
            arquivos_csv = {"ideal": os.path.join(folder, f"Ideal_{i}_{key}.csv".replace("ON_amfs", "ON_amfs_log"))}
            for modelo in modelos:
                arquivos_csv[modelo] = os.path.join(folder, f"{modelo}_{i}_{key}.csv".replace("ON_amfs", "ON_amfs_log"))

            dfs = {}
            for nome, caminho in arquivos_csv.items():
                if os.path.exists(caminho):
                    df = pd.read_csv(caminho)
                    if config["column_y"] in df.columns:
                        dfs[nome] = df

            if len(dfs) < len(modelos) + 1:
                continue

            ideal_vals = dfs["ideal"][config["column_y"]].tolist()
            min_len = min(len(ideal_vals), *(len(dfs[m][config["column_y"]]) for m in modelos))

            for t in range(min_len):
                for modelo in modelos:
                    val = dfs[modelo][config["column_y"]][t]
                    erro = int(round(val - ideal_vals[t]))  # arredondar para contagem
                    erros.append({
                        "Cluster": i,
                        "Modelo": modelo,
                        "Erro": erro
                    })

    return pd.DataFrame(erros)

def gerar_relatorio_resumido(df_erros):
    relatorio = df_erros.groupby(['Cluster', 'Modelo', 'Erro']).size().unstack(fill_value=0)
    relatorio = relatorio.sort_index(axis=1)
    return relatorio

# Processar
df_erros = calcular_erros(file_config, i_range, modelos_detectados)
relatorio_resumido = gerar_relatorio_resumido(df_erros)

# Salvar
relatorio_resumido.to_csv("relatorio_resumido_erros.csv")
print("Relatório resumido salvo em: relatorio_resumido_erros.csv")


Relatório resumido salvo em: relatorio_resumido_erros.csv
